# Solar Energy Consumption and Production Analysis

In this notebook, we analyze the building's energy consumption and PV energy production.

### Step 1: Import Libraries and Custom Analysis Script


In [1]:
# Solar Energy Consumption and Production Analysis

# Import necessary modules and custom script
import os
import sys
import dash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import webbrowser
import time
import traceback

# Set the directory explicitly if needed
os.chdir(r"F:\Master Study\Study Material\M2 FME Data\Arbaz Data\Green\Codes for project-20241121T133218Z-001\Codes for project\Milestone1 code")

# Add the current directory to sys.path so Python knows where to find the module
sys.path.append(r"F:\Master Study\Study Material\M2 FME Data\Arbaz Data\Green\Codes for project-20241121T133218Z-001\Codes for project\Milestone1 code")

# Import the custom solar analysis script
import solar_analysis_script as sas


C:\Users\ThinkPad T470\AppData\Local\Temp\ipykernel_11716\1472235167.py:8: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\ThinkPad T470\AppData\Local\Temp\ipykernel_11716\1472235167.py:9: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


### Step 2: Load and Filter the Data


In [2]:
# Define parameters
file_path = "Ense3buildingconsumption.csv"
start_date = '2023-09-25 00:00:00'
end_date = '2024-09-25 00:00:00'

# Load and filter data using the functions from the custom script
filtered_data = sas.load_data(file_path, start_date, end_date)

Loading data from file:  Ense3buildingconsumption.csv
Data successfully loaded and filtered.


f:\Master Study\Study Material\M2 FME Data\Arbaz Data\Green\Codes for project-20241121T133218Z-001\Codes for project\Milestone1 code\solar_analysis_script.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data['Consumption'] = (
f:\Master Study\Study Material\M2 FME Data\Arbaz Data\Green\Codes for project-20241121T133218Z-001\Codes for project\Milestone1 code\solar_analysis_script.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data.dropna(subset=['Consumption'], inplace=True)


In [3]:
# Constants
converter_efficiency = 0.90
global_irradiance = 4560
NOCT = 43
temperature_coefficient = 0.003
monthly_solar_energy = {1: 2.35, 2: 3.53, 3: 5.02, 4: 5.48, 5: 5.61, 6: 6.16, 7: 6.55, 8: 6.14, 9: 5.4, 10: 3.91, 11: 2.5, 12: 2.02}
monthly_temperatures = {1: 5.2, 2: 6.3, 3: 10.2, 4: 13.5, 5: 17.2, 6: 21.2, 7: 22.6, 8: 22.4, 9: 18, 10: 15.3, 11: 9.8, 12: 5.9}
initial_soc = 50

### Scenario 1: Without Battery Storage


In [4]:
print("### Scenario 1: Without Battery Storage ###")


### Scenario 1: Without Battery Storage ###


### Step 2: Plot Power Consumption Over Time (Scenario 1)


In [5]:
# Plot Power vs. Time for the Entire Year
fig = go.Figure()
fig.add_trace(go.Scatter(x=filtered_data.index, y=filtered_data['Consumption'],
                         mode='lines', name='Hourly Consumption'))

fig.update_layout(
    title='Power vs. Time for the Entire Year',
    xaxis_title='Date',
    yaxis_title='Power Consumption (kWh)',
    template='plotly_white'
)
fig.show()

### Step 3: Plot Average Daily Load Curve along with PV Production (Scenario 1)


In [6]:
# Create dropdown options for each month to visualize PV production and peak daily load

dropdown_options_no_battery = []

for month in range(1, 13):
    # Filter data for the specific month
    monthly_data = filtered_data[filtered_data.index.month == month]
    
    # Resample and calculate the peak load for each hour in the month (0-23)
    avg_hourly_load = monthly_data.resample('H').mean()
    peak_daily_load_month = avg_hourly_load.groupby(avg_hourly_load.index.hour).max()

    # Calculate PV production for the specified month
    monthly_pv_production = sas.calculate_monthly_pv_production(month, sas.number_of_panels)

    # Calculate unmet demand and curtailed energy
    unmet_demand = peak_daily_load_month['Consumption'] - monthly_pv_production
    unmet_demand[unmet_demand < 0] = 0  # No unmet demand if PV exceeds consumption

    energy_curtailed = monthly_pv_production - peak_daily_load_month['Consumption']
    energy_curtailed[energy_curtailed < 0] = 0  # No curtailment if consumption exceeds PV production

    # Add dropdown option for each month
    dropdown_options_no_battery.append({
        'label': f'{pd.to_datetime(month, format="%m").strftime("%B")}',  # Month name
        'method': 'update',
        'args': [
            {'y': [
                monthly_pv_production, 
                unmet_demand,
                -energy_curtailed,  # Show excess energy as a negative bar
                peak_daily_load_month['Consumption']
            ]},
            {'title': f'Solar PV Production, Unmet Demand, Excess Energy, and Electricity Demand - {pd.to_datetime(month, format="%m").strftime("%B")}'}
        ]
    })

# Initial figure with January data for non-battery scenario
# Filter data for January
january_data = filtered_data[filtered_data.index.month == 1]
avg_hourly_load_january = january_data.resample('H').mean()
peak_daily_load_january = avg_hourly_load_january.groupby(avg_hourly_load_january.index.hour).max()
monthly_pv_production_january = sas.calculate_monthly_pv_production(1, sas.number_of_panels)

# Calculate unmet demand and curtailed energy for January
unmet_demand_january = peak_daily_load_january['Consumption'] - monthly_pv_production_january
unmet_demand_january[unmet_demand_january < 0] = 0

energy_curtailed_january = monthly_pv_production_january - peak_daily_load_january['Consumption']
energy_curtailed_january[energy_curtailed_january < 0] = 0

# Plot Solar PV Production, Unmet Demand, Excess Energy, and Electricity Demand with dropdown options
fig = go.Figure()

# Add bar trace for Solar PV Production (positive values)
fig.add_trace(go.Bar(
    x=monthly_pv_production_january.index,
    y=monthly_pv_production_january,
    name='Solar PV Production (kW)',
    marker_color='orange'
))

# Add bar trace for Unmet Demand (positive values)
fig.add_trace(go.Bar(
    x=unmet_demand_january.index,
    y=unmet_demand_january,
    name='Unmet Demand (kW)',
    marker_color='red'
))

# Add bar trace for Excess Energy (negative values)
fig.add_trace(go.Bar(
    x=energy_curtailed_january.index,
    y=-energy_curtailed_january,  # Represent excess energy as negative values
    name='Excess Energy (kW)',
    marker_color='green'
))

# Add line trace for Electricity Demand
fig.add_trace(go.Scatter(
    x=peak_daily_load_january.index,
    y=peak_daily_load_january['Consumption'],
    mode='lines',
    name='Electricity Demand (kW)',
    line=dict(color='blue', dash='dash')
))

# Update layout to include dropdown menu on the right side of the plot under the legend
fig.update_layout(
    title='Solar PV Production, Unmet Demand, Excess Energy, and Electricity Demand - January',
    xaxis_title='Hour of the Day',
    yaxis_title='Energy (kWh)',
    xaxis=dict(tickvals=list(range(0, 24)), ticktext=[f'{h}:00' for h in range(0, 24)]),
    template='plotly_white',
    barmode='relative',  # Stack bars such that excess is shown in the negative direction
    updatemenus=[{
        'buttons': dropdown_options_no_battery,
        'direction': 'down',
        'showactive': True,
        'x': 1.15,  # Positioned to the right of the plot
        'xanchor': 'left',
        'y': 1.2,  # Positioned below the legend
        'yanchor': 'top',
    }]
)

# Show the figure
fig.show()

Calculating monthly PV production for month: 1 with 630 panels
Calculating monthly PV production for month: 2 with 630 panels
Calculating monthly PV production for month: 3 with 630 panels
Calculating monthly PV production for month: 4 with 630 panels
Calculating monthly PV production for month: 5 with 630 panels
Calculating monthly PV production for month: 6 with 630 panels
Calculating monthly PV production for month: 7 with 630 panels
Calculating monthly PV production for month: 8 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 10 with 630 panels
Calculating monthly PV production for month: 11 with 630 panels
Calculating monthly PV production for month: 12 with 630 panels
Calculating monthly PV production for month: 1 with 630 panels


### Step 4: Calculate and Display Annual Metrics (Scenario 1)


In [7]:
# Calculate Annual Metrics
annual_metrics = sas.calculate_annual_metrics(filtered_data, sas.number_of_panels)

# Display Annual Metrics
print("Annual PV Produced Energy (MWh):", annual_metrics["annual_pv_produced_kwh"])
print("Annual Imported Energy from Grid (MWh):", annual_metrics["annual_imported_kwh"])
print("Annual Exported Energy to Grid (MWh):", annual_metrics["annual_exported_kwh"])
print("Annual Total Energy Consumed (MWh):", annual_metrics["total_energy_consumed"])
print("Annual Total Energy Delivered to Load (MWh):", annual_metrics["energy_delivered_to_load"])
print("Performance Indicator (%):", annual_metrics["performance_indicator"])

Calculating annual metrics without battery storage...
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
C

### Step 5: Monthly Import/Export Analysis (Scenario 1)


In [8]:
# Calculate Monthly Import/Export
import_kwh, export_kwh = sas.monthly_import_export(filtered_data)

# Create Bar chart for monthly import/export
fig_import_export = go.Figure(data=[go.Bar(name='Import (kWh)', x=[f'Month {i+1}' for i in range(12)], y=import_kwh, marker_color='red'),
                                     go.Bar(name='Export (kWh)', x=[f'Month {i+1}' for i in range(12)], y=export_kwh, marker_color='green')])
fig_import_export.update_layout(barmode='group', title='Monthly Energy Import and Export', xaxis_title='Month', yaxis_title='Energy (kWh)')

# Show Bar chart
fig_import_export.show()


Calculating monthly PV production for month: 1 with 630 panels
Calculating monthly PV production for month: 2 with 630 panels
Calculating monthly PV production for month: 3 with 630 panels
Calculating monthly PV production for month: 4 with 630 panels
Calculating monthly PV production for month: 5 with 630 panels
Calculating monthly PV production for month: 6 with 630 panels
Calculating monthly PV production for month: 7 with 630 panels
Calculating monthly PV production for month: 8 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 10 with 630 panels
Calculating monthly PV production for month: 11 with 630 panels
Calculating monthly PV production for month: 12 with 630 panels


### Scenario 2: With Battery Storage


### Step 6: Calculate and Display Annual Metrics (Scenario 2 with Battery)


In [9]:
# Calculate Annual Metrics with Battery Storage
annual_metrics_battery = sas.calculate_annual_metrics_with_battery(filtered_data, sas.number_of_panels, sas.battery_capacity, sas.initial_soc)

# Display Annual Metrics for Scenario 2
print("Annual PV Produced Energy (MWh):", annual_metrics_battery["annual_pv_produced_kwh"])
print("Annual Storage Energy Supplied (MWh):", annual_metrics_battery["annual_storage_supplied_kwh"])
print("Annual Storage Consumed (charging by PV) (MWh):", annual_metrics_battery["annual_storage_consumed_kwh"])
print("Annual Imported Energy from Grid (MWh):", annual_metrics_battery["annual_imported_kwh"])
print("Annual Exported Energy to Grid (MWh):", annual_metrics_battery["annual_exported_kwh"])
print("Annual Total Energy Consumed (MWh):", annual_metrics_battery["total_energy_consumed"])
print("Annual Total Energy Delivered to Load (MWh):", annual_metrics_battery["energy_delivered_to_load"])
print("Performance Indicator (%):", annual_metrics_battery["performance_indicator"])

Calculating annual metrics with battery storage...


Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 9 with 63

### Step 7: Monthly Import/Export Analysis (Scenario 2)


In [10]:
# Calculate Monthly Import/Export with Battery
import_kwh_battery, export_kwh_battery = sas.monthly_import_export(filtered_data)

# Create Bar chart for monthly import/export (Scenario 2)
fig_import_export_battery = go.Figure(data=[go.Bar(name='Import (kWh)', x=[f'Month {i+1}' for i in range(12)], y=import_kwh_battery, marker_color='red'),
                                            go.Bar(name='Export (kWh)', x=[f'Month {i+1}' for i in range(12)], y=export_kwh_battery, marker_color='green')])
fig_import_export_battery.update_layout(barmode='group', title='Monthly Energy Import and Export (Scenario 2)', xaxis_title='Month', yaxis_title='Energy (kWh)')

# Show Bar chart for Scenario 2
fig_import_export_battery.show()


Calculating monthly PV production for month: 1 with 630 panels
Calculating monthly PV production for month: 2 with 630 panels
Calculating monthly PV production for month: 3 with 630 panels
Calculating monthly PV production for month: 4 with 630 panels
Calculating monthly PV production for month: 5 with 630 panels
Calculating monthly PV production for month: 6 with 630 panels
Calculating monthly PV production for month: 7 with 630 panels
Calculating monthly PV production for month: 8 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 10 with 630 panels
Calculating monthly PV production for month: 11 with 630 panels
Calculating monthly PV production for month: 12 with 630 panels


### Step 8: Plot Average Daily Load Curve along with PV Production (Scenario 2)


In [11]:
# Create dropdown options for each month to visualize PV production with battery storage and peak daily load
dropdown_options_battery = []
for month in range(1, 13):
    # Filter data for the specific month
    monthly_data = filtered_data[filtered_data.index.month == month]
    
    # Resample and calculate the peak load for each hour in the month
    avg_hourly_load = monthly_data.resample('H').mean()
    peak_daily_load_month = avg_hourly_load.groupby(avg_hourly_load.index.hour).max()

    # Calculate PV production for the specified month for the scenario with battery
    monthly_pv_production_with_battery = sas.calculate_monthly_pv_production(month, sas.number_of_panels)

    # Add dropdown option
    dropdown_options_battery.append({
        'label': f'{pd.to_datetime(month, format="%m").strftime("%B")}',  # Month name
        'method': 'update',
        'args': [
            {'y': [peak_daily_load_month['Consumption'], monthly_pv_production_with_battery]},
            {'title': f'Peak Daily Load Curve vs PV Production (Scenario 2 with Battery) - {pd.to_datetime(month, format="%m").strftime("%B")}'}
        ]
    })

# Initial figure with January data for Scenario 2 (with Battery)
# Filter data for January
january_data = filtered_data[filtered_data.index.month == 1]
avg_hourly_load_january = january_data.resample('H').mean()
peak_daily_load_january = avg_hourly_load_january.groupby(avg_hourly_load_january.index.hour).max()
monthly_pv_production_with_battery_january = sas.calculate_monthly_pv_production(1, sas.number_of_panels)

# Plot Peak Daily Load Curve and PV Production with dropdown options for Scenario 2 (with Battery)
fig = go.Figure()
fig.add_trace(go.Scatter(x=peak_daily_load_january.index, y=peak_daily_load_january['Consumption'],
                         mode='lines+markers', name='Peak Consumption'))
fig.add_trace(go.Scatter(x=monthly_pv_production_with_battery_january.index, y=monthly_pv_production_with_battery_january,
                         mode='lines+markers', name='PV Production (with Battery)', line=dict(color='orange')))

# Update layout to include dropdown menu on the right side of the plot under the legend
fig.update_layout(
    title='Peak Daily Load Curve vs PV Production (Scenario 2 with Battery) - January',
    xaxis_title='Hour of the Day',
    yaxis_title='Energy (kWh)',
    xaxis=dict(tickvals=list(range(0, 24)), ticktext=[f'{h}:00' for h in range(0, 24)]),
    template='plotly_white',
    updatemenus=[{
        'buttons': dropdown_options_battery,
        'direction': 'down',
        'showactive': True,
        'x': 1.15,  # Positioned to the right of the plot
        'xanchor': 'left',
        'y': 0.8,  # Positioned below the legend
        'yanchor': 'top',
    }]
)

# Show the figure
fig.show()

Calculating monthly PV production for month: 1 with 630 panels
Calculating monthly PV production for month: 2 with 630 panels
Calculating monthly PV production for month: 3 with 630 panels
Calculating monthly PV production for month: 4 with 630 panels
Calculating monthly PV production for month: 5 with 630 panels
Calculating monthly PV production for month: 6 with 630 panels
Calculating monthly PV production for month: 7 with 630 panels
Calculating monthly PV production for month: 8 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 10 with 630 panels
Calculating monthly PV production for month: 11 with 630 panels
Calculating monthly PV production for month: 12 with 630 panels
Calculating monthly PV production for month: 1 with 630 panels


In [12]:
# Create dropdown options for each month to visualize PV production with battery storage and peak daily load
dropdown_options_battery = []

for month in range(1, 13):
    # Filter data for the specific month
    monthly_data = filtered_data[filtered_data.index.month == month]
    
    # Resample and calculate the peak load for each hour in the month (0-23)
    avg_hourly_load = monthly_data.resample('H').mean()
    peak_daily_load_month = avg_hourly_load.groupby(avg_hourly_load.index.hour).max()

    # Calculate PV production for the specified month for the scenario with battery
    monthly_pv_production_with_battery = sas.calculate_monthly_pv_production(month, sas.number_of_panels)

    # Battery logic: calculate unmet demand, battery charging, and discharging
    battery_soc = sas.initial_soc  # Start with initial state of charge (SOC)
    energy_discharged = []
    energy_charged = []
    unmet_demand = []
    for hour in range(24):
        consumption = peak_daily_load_month['Consumption'].iloc[hour]
        pv_production = monthly_pv_production_with_battery[hour]

        if pv_production >= consumption:
            # Excess energy available
            excess_energy = pv_production - consumption

            if battery_soc < 100:
                charge = min(excess_energy * sas.converter_efficiency, (100 - battery_soc) / 100 * sas.battery_capacity)
                energy_charged.append(charge)
                battery_soc += (charge / sas.battery_capacity) * 100
            else:
                # No charging if the battery is full
                energy_charged.append(0)
            energy_discharged.append(0)
            unmet_demand.append(0)
        else:
            # PV production is not enough to meet demand
            deficit = consumption - pv_production
            if battery_soc > 20:
                discharge = min(deficit, (battery_soc - 20) / 100 * sas.battery_capacity)
                energy_discharged.append(discharge)
                battery_soc -= (discharge / sas.battery_capacity) * 100
                unmet_demand.append(deficit - discharge)
            else:
                # Not enough energy available in the battery
                energy_discharged.append(0)
                unmet_demand.append(deficit)
            energy_charged.append(0)

    # Add dropdown option for each month
    dropdown_options_battery.append({
        'label': f'{pd.to_datetime(month, format="%m").strftime("%B")}',  # Month name
        'method': 'update',
        'args': [
            {'y': [
                monthly_pv_production_with_battery, 
                unmet_demand,
                -pd.Series(energy_charged),  # Show energy charged as negative
                energy_discharged,
                peak_daily_load_month['Consumption']
            ]},
            {'title': f'Solar PV Production, Unmet Demand, Energy Discharged, Energy Charged, and Electricity Demand - {pd.to_datetime(month, format="%m").strftime("%B")}'}
        ]
    })

# Initial figure with January data for Scenario 2 (with Battery)
# Filter data for January
january_data = filtered_data[filtered_data.index.month == 1]
avg_hourly_load_january = january_data.resample('H').mean()
peak_daily_load_january = avg_hourly_load_january.groupby(avg_hourly_load_january.index.hour).max()
monthly_pv_production_with_battery_january = sas.calculate_monthly_pv_production(1, sas.number_of_panels)

# Battery logic for January
battery_soc = sas.initial_soc  # Start with initial SOC
energy_discharged_january = []
energy_charged_january = []
unmet_demand_january = []
for hour in range(24):
    consumption = peak_daily_load_january['Consumption'].iloc[hour]
    pv_production = monthly_pv_production_with_battery_january[hour]

    if pv_production >= consumption:
        # Excess energy available
        excess_energy = pv_production - consumption

        if battery_soc < 100:
            charge = min(excess_energy * sas.converter_efficiency, (100 - battery_soc) / 100 * sas.battery_capacity)
            energy_charged_january.append(charge)
            battery_soc += (charge / sas.battery_capacity) * 100
        else:
            # No charging if the battery is full
            energy_charged_january.append(0)
        energy_discharged_january.append(0)
        unmet_demand_january.append(0)
    else:
        # PV production is not enough to meet demand
        deficit = consumption - pv_production
        if battery_soc > 20:
            discharge = min(deficit, (battery_soc - 20) / 100 * sas.battery_capacity)
            energy_discharged_january.append(discharge)
            battery_soc -= (discharge / sas.battery_capacity) * 100
            unmet_demand_january.append(deficit - discharge)
        else:
            # Not enough energy available in the battery
            energy_discharged_january.append(0)
            unmet_demand_january.append(deficit)
        energy_charged_january.append(0)

# Plot Solar PV Production, Unmet Demand, Energy Charged, Energy Discharged, and Electricity Demand with dropdown options
fig = go.Figure()

# Add bar trace for Solar PV Production (positive values)
fig.add_trace(go.Bar(
    x=monthly_pv_production_with_battery_january.index,
    y=monthly_pv_production_with_battery_january,
    name='Solar PV Production (kW)',
    marker_color='orange'
))

# Add bar trace for Unmet Demand (positive values)
fig.add_trace(go.Bar(
    x=january_data.index.hour.unique(),
    y=unmet_demand_january,
    name='Unmet Demand (kW)',
    marker_color='red'
))

# Add bar trace for Energy Charged to Battery (negative values)
fig.add_trace(go.Bar(
    x=january_data.index.hour.unique(),
    y=[-val for val in energy_charged_january],  # Represent energy charged to battery as negative values
    name='Energy Charged to Battery (kW)',
    marker_color='green'
))

# Add bar trace for Energy Discharged from Battery (positive values)
fig.add_trace(go.Bar(
    x=january_data.index.hour.unique(),
    y=energy_discharged_january,
    name='Energy Discharged from Battery (kW)',
    marker_color='purple'
))

# Add line trace for Electricity Demand
fig.add_trace(go.Scatter(
    x=peak_daily_load_january.index,
    y=peak_daily_load_january['Consumption'],
    mode='lines',
    name='Electricity Demand (kW)',
    line=dict(color='blue', dash='dash')
))

# Update layout to include dropdown menu on the right side of the plot under the legend
fig.update_layout(
    title='Solar PV Production, Unmet Demand, Energy Charged, Energy Discharged, and Electricity Demand - January',
    xaxis_title='Hour of the Day',
    yaxis_title='Energy (kWh)',
    xaxis=dict(tickvals=list(range(0, 24)), ticktext=[f'{h}:00' for h in range(0, 24)]),
    template='plotly_white',
    barmode='relative',  # Stack bars for better comparison
    updatemenus=[{
        'buttons': dropdown_options_battery,
        'direction': 'down',
        'showactive': True,
        'x': 1.15,  # Positioned to the right of the plot
        'xanchor': 'left',
        'y': 1.2,  # Positioned below the legend
        'yanchor': 'top',
    }]
)

# Show the figure
fig.show()

Calculating monthly PV production for month: 1 with 630 panels
Calculating monthly PV production for month: 2 with 630 panels
Calculating monthly PV production for month: 3 with 630 panels
Calculating monthly PV production for month: 4 with 630 panels
Calculating monthly PV production for month: 5 with 630 panels
Calculating monthly PV production for month: 6 with 630 panels
Calculating monthly PV production for month: 7 with 630 panels
Calculating monthly PV production for month: 8 with 630 panels
Calculating monthly PV production for month: 9 with 630 panels
Calculating monthly PV production for month: 10 with 630 panels
Calculating monthly PV production for month: 11 with 630 panels
Calculating monthly PV production for month: 12 with 630 panels
Calculating monthly PV production for month: 1 with 630 panels


In [ ]:
#a